In [ ]:

!pip install foolbox
import foolbox as fb
from google.colab import files

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime 
from tqdm.notebook import tqdm 
import statistics

# parameters
RANDOM_SEED = 42
BATCH_SIZE = 100
N_EPOCHS = 475
IMG_SIZE = 32
N_CLASSES = 10

LEARNING_RATE = 0.001
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-3


# define transforms
# transforms.ToTensor() automatically scales the images to [0,1] range
# transforms = transforms.Compose([transforms.Resize((32, 32)), transforms.Normalize(mean=[0.485, 0.456, 0.406], 
#                                 std=[0.229, 0.224, 0.225])], transforms.ToTensor()])

transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# transforms = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# download and create datasets
train_dataset = datasets.CIFAR10(root='cifar10_data', train=True, transform=transforms, download=True)

valid_dataset = datasets.CIFAR10(root='cifar10_data', train=False, transform=transforms, download=True)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

#Class labels
classes = ('Airplane', 'Car', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck')


#implementing AlexNet_Exact model
class AlexNet_Exact(nn.Module):

    def __init__(self):
        super(AlexNet_Exact, self).__init__()        

        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(6, 6))

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride=(1,1), padding=(2,2))
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=192, kernel_size=(3,3), stride=(1,1), padding=(2,2))
        self.conv3 = nn.Conv2d(in_channels=192, out_channels=384, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv4 = nn.Conv2d(in_channels=384, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride=(1,1), padding=(1,1))
                
        self.dropout = nn.Dropout()
        self.linear1 = nn.Linear(in_features = 256 * 6 * 6, out_features = 1024)
        self.linear2 = nn.Linear(in_features = 1024, out_features = 1024)
        self.linear3 = nn.Linear(in_features = 1024, out_features = 10)
        
        
    def forward(self, x):
        # Feature extractor
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        x = self.conv3(x)
        x = self.relu(x)

        x = self.conv4(x)
        x = self.relu(x)

        x = self.conv5(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.avgpool(x)
        
        x = x.view(x.size(0), 256 * 6 * 6)
        
        x = self.dropout(x)
        x = self.linear1(x)
        x = self.relu(x)
        
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        
        logits = self.linear3(x)
#         probs = F.softmax(logits, dim=1)
        
        return logits



torch.manual_seed(RANDOM_SEED)
model_exact = AlexNet_Exact()
optimizer = torch.optim.SGD(model_exact.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) 
criterion = nn.CrossEntropyLoss()

# Train model 
# model_exact, optimizer, _ = training_loop(model_exact, criterion, optimizer, train_loader, valid_loader, N_EPOCHS)



#Instantiating CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#Verifying CUDA
print(device)
#Move the input and AlexNet_model to GPU for speed if available
model_exact.to(device)

#Testing Accuracy
def get_accuracy(model, data_loader, device):
  correct = 0
  total = 0
  with torch.no_grad():
      for data in data_loader:
          images, labels = data[0].to(device), data[1].to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

  accuracy = (correct / total)
  return accuracy

print_every = 10
for epoch in range(N_EPOCHS):  # loop over the dataset multiple times
    running_loss = 0.0

    if epoch == 30:
        optimizer = torch.optim.SGD(model_exact.parameters(), lr=0.0005, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) 
        print("\nlearning rate decay to 1/10...")
    if epoch == 40:
        optimizer = torch.optim.SGD(model_exact.parameters(), lr=0.0001, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) 
        print("\nlearning rate decay to 1/100...")

    if epoch == 50:
        optimizer = torch.optim.SGD(model_exact.parameters(), lr=0.00005, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) 
        print("\nlearning rate decay to 1/10...")
    if epoch == 60:
        optimizer = torch.optim.SGD(model_exact.parameters(), lr=0.00001, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) 
        print("\nlearning rate decay to 1/100...")

    if epoch == 70:
        optimizer = torch.optim.SGD(model_exact.parameters(), lr=0.000005, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY) 
        print("\nlearning rate decay to 1/100...")

    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model_exact(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
    
    if epoch % print_every == (print_every - 1):
      epoch_loss = running_loss / len(train_loader.dataset)
      valid_acc = get_accuracy(model_exact, valid_loader, device)
      print(f'{datetime.now().time().replace(microsecond=0)} --- '
      f'Epoch: {epoch+1}\t'
      f'Train loss: {epoch_loss:.4f}\t'
      f'Valid accuracy: {100 * valid_acc:.2f}')


print('Finished Training of AlexNet')


filename = '../dataset/Cifar10.pth'
torch.save(model_exact.state_dict(), filename)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 17.9 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 181 kB 73.1 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting cifar10_data/cifar-10-python.tar.gz to cifar10_data
Files already downloaded and verified
cpu
16:38:28 --- Epoch: 10	Train loss: 0.3786	Valid accuracy: 80.89
20:14:08 --- Epoch: 20	Train loss: 0.2371	Valid accuracy: 80.96
23:52:13 --- Epoch: 30	Train loss: 0.1907	Valid accuracy: 78.75

learning rate decay to 1/10...
03:32:22 --- Epoch: 40	Train loss: 0.0576	Valid accuracy: 84.33

learning rate decay to 1/100...
07:15:57 --- Epoch: 50	Train loss: 0.0045	Valid accuracy: 86.71

learning rate decay to 1/10...
10:58:36 --- Epoch: 60	Train loss: 0.0049	Valid accuracy: 86.50

learning rate decay to 1/100...
